In [28]:
#Create_Skills_data_from_job_data
import json
Base_Skills=[] 
with open('data/jobs.json' , 'r' , encoding='UTF-8') as f :
    data=json.load(f)
    for job in data:
        if (job['skills']):
            List_skills=job['skills']
            for skill in List_skills:
                Base_Skills.append(skill)
    newdata={"SKILLS": Base_Skills}
    with open ('data/Skills_Data.json' , 'w' , encoding='UTF-8') as f:
        json.dump(newdata , f , ensure_ascii= False )


In [29]:
#Create_Skills_Function_from_dictionary
import spacy, re
from skweak import heuristics, gazetteers, generative, utils
tries=gazetteers.extract_json_data('data/Skills_Data.json')
nlp=spacy.load('en_core_web_md' , disable=['ner'])
gazetteer = gazetteers.GazetteerAnnotator("SKILLS", tries)
lf3= gazetteers.GazetteerAnnotator("SKILLS", tries)

Extracting data from data/Skills_Data.json

Populating trie for class SKILLS (number: 2394)


In [30]:
#Create_Function_Foe_Experience_Detection(Use Regex)
def experience_detector (doc):
    expression=r'[0-9][+] years'
    for match in re.finditer(expression, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end)
        if(span):
            yield span.start , span.end ,  "EXPERIENCE"
lf1 = heuristics.FunctionAnnotator("experience", experience_detector)
doc=lf1(nlp(' 2+ years of experience. 3+ years i worked with a lot of tools'))
doc.spans

{'experience': [2+ years, 3+ years]}

In [ ]:
#Create_Function_Foe_diploma_major(Use Regex)
def diploma_major_detector (doc):
    expression=re.compile(r"(^.*(Ph.D|MS|Master|BA|Bachelor|BS)\S*) in (\S*)")
    for match in re.finditer(expression, doc.text):
        start, end = match.span(3)
        span = doc.char_span(start, end)
        if(span):
            yield span.start , span.end ,  "DIPLOMA_MAJOR"

lf5 = heuristics.FunctionAnnotator("Diploma_major", diploma_major_detector)

In [62]:
#Use of a data to create a dicti
import pandas as pd
data= pd.read_csv("data.csv" , encoding='windows-1252')
data


,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,1,Dr. Ihsan Ullah,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,NaN,France,NaN,"P2P Networks, Telecommunication Systems",NaN
1,2,Dr. Atiq Ahmed,University of Balochistan,Computer Science & IT,Balochistan,Associate Professor,PhD,NaN,France,NaN,"Wireless Networks, Telecommunication Systems, ...",NaN
2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
3,4,Mr. Muhammad Khalid Badini,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,MS,NaN,NaN,NaN,Operating System & DBMS,On study leave
4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1976,1977,Dr. Masroor Hussain,Ghulam Ishaq Khan Institute,Computer Science and Engineering,KPK,Associate Professor,PhD,NaN,NaN,NaN,"High Performance Computing, Computational Scie...",NaN
1977,1978,Dr. Rashad M Jillani,Ghulam Ishaq Khan Institute,Computer Science and Engineering,KPK,Assistant Professor,PhD,Florida Atlantic University,USA,2012.0,"Digital Multimedia Systems, Video Compression ...",NaN
1978,1979,Dr. Asif Khan,Ghulam Ishaq Khan Institute,Computer Science and Engineering,KPK,Assistant Professor,PhD,NaN,Austria,NaN,"Unmanned Aerial Vehicle (UAV), Multirobot Syst...",NaN
1979,1980,Dr. Shahabuddin Ansari,Ghulam Ishaq Khan Institute,Computer Science and Engineering,KPK,Assistant Professor,PhD,Ghulam Ishaq Khan Institute of Science and Tec...,Pakistan,NaN,"Medical Image Processing and Analysis, Digital...",NaN


In [65]:
with open('Diploma_Dic.json' , 'r' , encoding='UTF-8') as f :
    DIPLOMA=json.load(f)

print(len(DIPLOMA))
with open ('Diploma_Major_Dic.json' ,encoding='UTF-8') as f :
    DIPLOMA_MAJOR=json.load(f)

print(len(DIPLOMA_MAJOR))

74

69


In [32]:
#Create Diploma_Function_with_Regex
def Diploma_fun(doc):
    for key in DIPLOMA:
                for match in re.finditer(key , doc.text , re.IGNORECASE):
                    start, end = match.span()
                    span = doc.char_span(start, end)
                    if(span):
                        yield (span.start , span.end ,  "DIPLOMA")
    
lf4 = heuristics.FunctionAnnotator("Diploma", Diploma_fun)

In [33]:
#Create_Diploma_Major_Function_with_Regex
def Diploma_major_fun(doc):  
    for key in DIPLOMA_MAJOR:
                for match in re.finditer(key , doc.text , re.IGNORECASE):
                    start, end = match.span()
                    span = doc.char_span(start, end)
                    if(span):
                        yield (span.start , span.end ,  "DIPLOMA_MAJOR")
    
lf2 = heuristics.FunctionAnnotator("Diploma_major", Diploma_major_fun)

In [35]:
#create_corpus_annotated_to_train_the_model
docs=[]
with open('Corpus.txt' , 'r') as f :
    data=f.readlines()
    for text in data:
        if (len(text) !=1):
            newtext=str(text)
            doc=nlp(newtext)
            doc=lf1(lf2(lf3(lf4(lf5(doc)))))
            print(doc.spans)
            docs.append(doc)



{'Diploma': [PhD, PhD or MSc], 'SKILLS': [systems, hardware, technologies, technology, technology, Computer, Computer, system, systems, Linux, C++, Python, software, hardware], 'Diploma_major': [Computer science, Computer engineering, Electrical engineering, Computer science, Electrical engineering, Computer engineering, machine learning, machine learning, Electrical, Electrical engineering, engineering, engineering, Computer science, Computer engineering, engineering, engineering], 'experience': []}

{'Diploma': [Bachelors], 'SKILLS': [developing, software, Linux, software, Linux, C++, Java, Python, Development, deployment, applications, Docker, technical], 'Diploma_major': [Computer Science, Computer Science, Computer Science, technical], 'experience': []}

{'Diploma': [BA, BS, BA/BS], 'SKILLS': [technical, Android, iOS, software, BA, real-time, technologies], 'Diploma_major': [Computer Science, Computer Science, engineering, Computer Science, software engineering, technical, enginee

In [37]:
from skweak import aggregation
model = aggregation.HMM("hmm", ["DIPLOMA", "DIPLOMA_MAJOR" , "EXPERIENCE" , "SKILLS"])

In [38]:
docs = model.fit_and_aggregate(docs)
utils.display_entities(docs[16], "hmm", add_tooltip=True) 

Starting iteration 1

Finished E-step with 120 documents

Starting iteration 2


         1       -9249.9450             +nan


Finished E-step with 120 documents

Starting iteration 3


         2       -8938.4886        +311.4564


Finished E-step with 120 documents

Starting iteration 4


         3       -8915.9088         +22.5798


Finished E-step with 120 documents


         4       -8906.2172          +9.6916


In [39]:
for doc in docs:
    doc.ents = doc.spans["hmm"]
utils.docbin_writer(docs, "train.spacy")

Write to train.spacy...done


In [41]:
#Train_The_Model_with_Train_data_created_with_skweak(automatic_annotation)
#577 itérations
!python -m spacy train config.cfg --output ./output --paths.train train.spacy --paths.dev train.spacy

^C


Model _ Evaluation : 
UBIAI_MODEL VS Skweak_Model

UBIAI MODEL EVALUATION

In [9]:
!python -m spacy evaluate UBIAI-Model/model-best/ UBIAI_Test_Data.spacy

ℹ Using CPU



================================== Results ==================================



TOK     100.00

NER P   74.27 

NER R   80.10 

NER F   77.08 

SPEED   4506  





=============================== NER (per type) ===============================



                    P       R       F

DIPLOMA         85.71   66.67   75.00

DIPLOMA_MAJOR   33.33   16.67   22.22

EXPERIENCE      81.82   81.82   81.82

SKILLS          74.05   83.03   78.29




Skweak Model EVALUATION

In [43]:
!python -m spacy evaluate output/model-best/ UBIAI_Test_Data.spacy

ℹ Using CPU



================================== Results ==================================



TOK     100.00

NER P   31.78 

NER R   17.80 

NER F   22.82 

SPEED   2711  





=============================== NER (per type) ===============================



                     P       R       F

DIPLOMA          33.33   22.22   26.67

DIPLOMA_MAJOR    14.29   50.00   22.22

EXPERIENCE      100.00   27.27   42.86

SKILLS           33.77   15.76   21.49




WEAK_ANNOTATION